In [271]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
from shapely.geometry import shape, Point
import plotly.express as px

In [272]:
path = Path('data') / 'Motor_Vehicle_Collisions_-_Crashes.csv'
print(path)
df = pd.read_csv(path, parse_dates=[['CRASH DATE', 'CRASH TIME']], na_values=['nan', 'Unspecified'])
df.head()

data\Motor_Vehicle_Collisions_-_Crashes.csv


C:\Users\gevia\AppData\Local\Temp\ipykernel_22100\2342807764.py:3: DtypeWarning:

Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.



,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,NaN,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN


In [273]:
clean_df = df.drop(['COLLISION_ID', 'CROSS STREET NAME', 'OFF STREET NAME', 'LOCATION', 'ZIP CODE'], axis=1)
clean_df = clean_df.dropna(subset=['LATITUDE', 'LONGITUDE'])
clean_df.rename(columns={
    'CRASH DATE_CRASH TIME': 'DATETIME', 
    'ON STREET NAME': 'STREET',
    }, inplace=True)
clean_df.columns = [col.replace('NUMBER OF ', '').replace('CONTRIBUTING ', '').replace('CODE ', '') for col in clean_df.columns]
clean_df.head()

,DATETIME,BOROUGH,LATITUDE,LONGITUDE,STREET,PERSONS INJURED,PERSONS KILLED,PEDESTRIANS INJURED,PEDESTRIANS KILLED,CYCLIST INJURED,...,FACTOR VEHICLE 1,FACTOR VEHICLE 2,FACTOR VEHICLE 3,FACTOR VEHICLE 4,FACTOR VEHICLE 5,VEHICLE TYPE 1,VEHICLE TYPE 2,VEHICLE TYPE 3,VEHICLE TYPE 4,VEHICLE TYPE 5
3,2021-09-11 09:35:00,BROOKLYN,40.667202,-73.866500,NaN,0.0,0.0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,40.683304,-73.917274,SARATOGA AVENUE,0.0,0.0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2021-12-14 17:05:00,NaN,40.709183,-73.956825,BROOKLYN QUEENS EXPRESSWAY,0.0,0.0,0,0,0,...,Passing Too Closely,NaN,NaN,NaN,NaN,Sedan,Tractor Truck Diesel,NaN,NaN,NaN
7,2021-12-14 08:17:00,BRONX,40.868160,-73.831480,NaN,2.0,0.0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,Sedan,Sedan,NaN,NaN,NaN
8,2021-12-14 21:10:00,BROOKLYN,40.671720,-73.897100,NaN,0.0,0.0,0,0,0,...,Driver Inexperience,NaN,NaN,NaN,NaN,Sedan,NaN,NaN,NaN,NaN


In [274]:
factor_cols = [col for col in clean_df.columns if 'FACTOR VEHICLE' in col]
vehicle_type_cols = [col for col in clean_df.columns if 'VEHICLE TYPE' in col]
clean_df[factor_cols+vehicle_type_cols] = clean_df[factor_cols+vehicle_type_cols].apply(
    lambda x: x.str.capitalize().replace(' / ', '/').replace('Station wagon/sport utility vehicle', 'Suv')
    )

In [275]:
rep_dict = {
    'Illnes': 'Illness',
    'Drugs (illegal)': 'Drugs (Illegal)',
    'Cell Phone (hand-Held)': 'Cell Phone (hand-held)',
}
clean_df[factor_cols] = clean_df[factor_cols].replace(rep_dict)
clean_df['BOROUGH'] = clean_df['BOROUGH'].str.title()

In [276]:
clean_df[factor_cols+vehicle_type_cols] = clean_df[factor_cols+vehicle_type_cols].fillna('')

In [283]:
clean_df['FACTORS'] = clean_df.apply(lambda x: [x[col] for col in factor_cols if x[col]], axis=1)
clean_df['VEHICLE TYPES'] = clean_df.apply(lambda x: [x[col] for col in vehicle_type_cols if x[col]], axis=1)
clean_df['N VEHICLES'] = clean_df['VEHICLE TYPES'].apply(len)
clean_df['SERIOUSNESS'] = clean_df.apply(lambda x: 'Killed' if x['PERSONS KILLED'] else ('Injured' if x['PERSONS INJURED'] else 'No victims'), axis=1)

clean_df = clean_df.drop(factor_cols + vehicle_type_cols, axis=1)
clean_df = clean_df.drop('BOROUGH', axis=1)
clean_df.head()

,DATETIME,LATITUDE,LONGITUDE,STREET,PERSONS INJURED,PERSONS KILLED,PEDESTRIANS INJURED,PEDESTRIANS KILLED,CYCLIST INJURED,CYCLIST KILLED,MOTORIST INJURED,MOTORIST KILLED,FACTORS,VEHICLE TYPES,N VEHICLES,SERIOUSNESS
3,2021-09-11 09:35:00,40.667202,-73.866500,NaN,0.0,0.0,0,0,0,0,0,0,[],[Sedan],1,No victims
4,2021-12-14 08:13:00,40.683304,-73.917274,SARATOGA AVENUE,0.0,0.0,0,0,0,0,0,0,[],[],0,No victims
6,2021-12-14 17:05:00,40.709183,-73.956825,BROOKLYN QUEENS EXPRESSWAY,0.0,0.0,0,0,0,0,0,0,[Passing too closely],"[Sedan, Tractor truck diesel]",2,No victims
7,2021-12-14 08:17:00,40.868160,-73.831480,NaN,2.0,0.0,0,0,0,0,2,0,[],"[Sedan, Sedan]",2,Injured
8,2021-12-14 21:10:00,40.671720,-73.897100,NaN,0.0,0.0,0,0,0,0,0,0,[Driver inexperience],[Sedan],1,No victims


In [284]:
with open(Path('data') / 'new-york-city-boroughs.geojson', 'r') as f:
    nyc_boroughs = json.load(f)
areas_names= [nyc_boroughs['features'][i]['properties']['name'] for i in range(len(nyc_boroughs['features']))]
print(f"Areas in geojson: {areas_names}")

with open(Path('data') / 'nyc-neighborhoods.geo.json', 'r') as f:
    nyc_districts = json.load(f)
areas_names= [nyc_districts['features'][i]['properties']['name'] for i in range(len(nyc_districts['features']))]
print(f"Areas in geojson: {areas_names}")


Areas in geojson: ['Staten Island', 'Queens', 'Brooklyn', 'Manhattan', 'Bronx']
Areas in geojson: ['Todt Hill-Emerson Hill-Heartland Village-Lighthouse Hill', 'New Springville-Bloomfield-Travis', 'Westerleigh', 'Breezy Point-Belle Harbor-Rockaway Park-Broad Channel', 'Hammels-Arverne-Edgemere', 'Far Rockaway-Bayswater', 'Woodhaven', 'Lindenwood-Howard Beach', 'Ozone Park', 'South Ozone Park', 'Bellerose', 'Glen Oaks-Floral Park-New Hyde Park', 'Oakland Gardens', 'Richmond Hill', 'Jamaica Estates-Holliswood', 'Pomonok-Flushing Heights-Hillcrest', 'Briarwood-Jamaica Hills', 'Kew Gardens Hills', 'Jamaica', 'Queens Village', 'Fresh Meadows-Utopia', 'Douglas Manor-Douglaston-Little Neck', 'Bayside-Bayside Hills', 'Auburndale', 'East Flushing', 'Queensboro Hill', 'Flushing', 'Whitestone', 'Ft. Totten-Bay Terrace-Clearview', 'Murray Hill', 'Kew Gardens', 'Forest Hills', 'College Point', 'Ridgewood', 'Glendale', 'Rego Park', 'Middle Village', 'Hollis', 'South Jamaica', 'St. Albans', 'Springfie

## Add boroughs and districts to the dataframe

In [285]:
nyc_borough_polys = {feature['properties']['name']: shape(feature['geometry']) for feature in nyc_boroughs['features']}
nyc_district_polys = {feature['properties']['name']: shape(feature['geometry']) for feature in nyc_districts['features']}

def find_borough(lognitude, latitude, polys):
    point = Point(lognitude, latitude)
    for borough, poly in polys.items():
        if poly.contains(point):
            return borough
    return 'Unknown'

In [286]:
final_df = clean_df.copy()
final_df['Borough'] = final_df.apply(lambda x: find_borough(x['LONGITUDE'], x['LATITUDE'], nyc_borough_polys), axis=1)
final_df['District'] = final_df.apply(lambda x: find_borough(x['LONGITUDE'], x['LATITUDE'], nyc_district_polys), axis=1)
final_df = final_df[(final_df['Borough'] != 'Unknown') & (final_df['District'] != 'Unknown')]

# Export cleaned data to csv

In [301]:
final_df.to_csv(Path('data') / 'cleaned_nyc_crashes.csv', index=False)

In [63]:
from ast import literal_eval
df = pd.read_csv(Path('data') / 'cleaned_nyc_crashes.csv', parse_dates=['DATETIME'], converters={"FACTORS": literal_eval})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1760727 entries, 0 to 1760726
Data columns (total 18 columns):
 #   Column               Dtype         
---  ------               -----         
 0   DATETIME             datetime64[ns]
 1   BOROUGH              object        
 2   LATITUDE             float64       
 3   LONGITUDE            float64       
 4   STREET               object        
 5   PERSONS INJURED      float64       
 6   PERSONS KILLED       float64       
 7   PEDESTRIANS INJURED  int64         
 8   PEDESTRIANS KILLED   int64         
 9   CYCLIST INJURED      int64         
 10  CYCLIST KILLED       int64         
 11  MOTORIST INJURED     int64         
 12  MOTORIST KILLED      int64         
 13  FACTORS              object        
 14  VEHICLE TYPES        object        
 15  N VEHICLES           int64         
 16  SERIOUSNESS          object        
 17  SERIOUSNESS SIZE     int64         
dtypes: datetime64[ns](1), float64(4), int64(8), object(5)
memory u